In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

#from preprocessing.data_cleaning import DataCleaner

pd.options.display.max_columns = 999

In [2]:
# LEITURA DOS DADOS BRUTOS ADQUIRIDOS 

df_raw_ = pd.read_parquet("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\machine_learning\\df_ml.parquet",use_pandas_metadata=False, engine="pyarrow")

df = df_raw_.copy()

In [4]:
df.head()

,dia_semana,ano,adversario,publico_pagante,publico_presente,renda_bruta,estadio,campeonato,placar,horario,dia,mes,data_formatada,campeonato_ajustado,gols_visitante,gols_mandante,gols_total,pontos_alcancados,resultado,aproveitamento_temporada_previo,jogos_invencibilidade_previo,estreia,classico,capacidade_estadio,ocupacao_estadio,dia_semana_int,Data,precipitacao_total,temp_media_bulbo_seco,temp_media_ponto_orvalho,velocidade_media_vento,precipitacao_total_lag1,precipitacao_total_lag2,precipitacao_total_lag3,precipitacao_total_lag4,precipitacao_total_lag5,precipitacao_total_lag6,precipitacao_total_lag7,precipitacao_total_media_sete_dias,temp_media_bulbo_seco_lag1,temp_media_bulbo_seco_lag2,temp_media_bulbo_seco_lag3,temp_media_bulbo_seco_lag4,temp_media_bulbo_seco_lag5,temp_media_bulbo_seco_lag6,temp_media_bulbo_seco_lag7,temp_media_bulbo_seco_media_sete_dias,temp_media_ponto_orvalho_lag1,temp_media_ponto_orvalho_lag2,temp_media_ponto_orvalho_lag3,temp_media_ponto_orvalho_lag4,temp_media_ponto_orvalho_lag5,temp_media_ponto_orvalho_lag6,temp_media_ponto_orvalho_lag7,temp_media_ponto_orvalho_media_sete_dias,velocidade_media_vento_lag1,velocidade_media_vento_lag2,velocidade_media_vento_lag3,velocidade_media_vento_lag4,velocidade_media_vento_lag5,velocidade_media_vento_lag6,velocidade_media_vento_lag7,velocidade_media_vento_media_sete_dias
0,domingo,2011,Cruzeiro,1243,<NA>,12820.00,Parque do Sabiá,Amistoso 2011,0 × 3,17:00,23,1,2011-01-23,Amistoso,0,3,3,0,Derrota,0.00,0,True,False,39900,0.031153,7,2011-01-23,0.0,24.141667,17.6500,2.937500,0.0,0.0,0.0,0.0,6.0,0.0,17.4,3.342857,24.908333,25.079167,24.958333,24.387500,24.654167,25.170833,22.000000,24.451190,17.554167,17.062500,16.770833,18.283333,19.262500,18.741667,19.712500,18.198214,3.050000,3.095833,1.545833,1.579167,2.100000,1.591667,0.762500,1.960714
1,domingo,2011,Caldense,4151,<NA>,82980.00,Arena do Jacaré,1ª rodada do Campeonato Mineiro 2011,3 × 0,17:00,30,1,2011-01-30,Campeonato Mineiro,3,0,3,0,Derrota,0.00,0,False,False,19998,0.207571,7,2011-01-30,0.0,26.137500,16.9125,1.425000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,25.279167,25.116667,24.979167,24.391667,23.895833,23.758333,24.141667,24.508929,17.491667,17.008333,16.754167,16.570833,15.887500,16.579167,17.650000,16.848810,1.983333,3.000000,3.112500,2.895833,2.383333,2.570833,2.937500,2.697619
2,domingo,2011,Cruzeiro,4144,<NA>,55230.00,Castor Cifuentes,2ª rodada do Campeonato Mineiro 2011,0 × 1,19:30,6,2,2011-02-06,Campeonato Mineiro,0,1,1,0,Derrota,0.00,0,False,False,<NA>,NaN,7,2011-02-06,0.0,24.987500,15.3125,2.820833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,24.745833,24.454167,24.912500,25.112500,26.029167,26.387500,26.137500,25.397024,14.370833,15.050000,15.354167,16.045833,16.416667,16.520833,16.912500,15.810119,3.104167,3.125000,3.050000,3.012500,1.900000,1.683333,1.425000,2.471429
3,sábado,2011,Atlético-MG,9793,<NA>,267256.88,Arena do Jacaré,3º rodada do Campeonato Mineiro 2011,3 × 4,19:30,12,2,2011-02-12,Campeonato Mineiro,3,4,7,3,Vitoria,0.00,0,False,True,19998,0.489699,6,2011-02-12,0.0,25.445833,16.4125,2.575000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,25.654167,25.920833,25.037500,24.616667,25.370833,24.987500,24.745833,25.190476,17.108333,15.604167,15.412500,14.895833,16.137500,15.312500,14.370833,15.548810,1.691667,2.608333,2.987500,3.145833,3.408333,2.820833,3.104167,2.823810
4,quarta-feira,2011,Estudiantes,10955,<NA>,377267.50,Arena do Jacaré,1ª Rodada - Fase de Grupos da Copa Libertadore...,5 × 0,22:00,16,2,2011-02-16,Libertadores,5,0,5,0,Derrota,0.25,1,False,False,19998,0.547805,3,2011-02-16,0.0,25.154167,16.5250,3.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,25.508333,25.020833,24.962500,25.445833,25.654167,25.920833,25.037500,25.364286,14.983333,15.937500,14.904167,16.412500,17.108333,15.604167,15.412500,15.766071,2.216667,2.729167,2.470833,2.575000,1.691667,2.608333,2.987500,2.468452


In [4]:
df['resultado_previo'] = df['resultado'].shift(1)
df['gols_mandante_previo'] = df['gols_mandante'].shift(1)
df['gols_visitante_previo'] = df['gols_visitante'].shift(1)
df['gols_total_previo'] = df['gols_total'].shift(1)
df['saldo_gols_previo'] = df['gols_mandante_previo'] - df['gols_visitante_previo']

def periodo_do_dia(horario):
    hora = int(horario.split(':')[0])
    if hora < 12:
        return 'manha'
    elif hora < 18:
        return 'tarde'
    else:
        return 'noite'

df['periodo'] = df['horario'].apply(periodo_do_dia)

In [5]:
df = df[df['adversario'] != 'Cruzeiro']

In [6]:
df = df.drop(df[df.publico_pagante.isna()].index, axis=0)

#df = df[~(df['publico_pagante'] == 0)]

In [7]:
df['publico_pagante_ultimo_jogo_em_casa'] = df['publico_pagante'].shift(1)
df['renda_bruta_ultimo_jogo_em_casa'] = df['renda_bruta'].shift(1)

df = df.drop(df[df['publico_pagante_ultimo_jogo_em_casa'].isna()].index, axis=0)

In [8]:
# df = df.drop(['publico_presente','placar','campeonato','data_formatada','renda_bruta','gols_visitante','gols_mandante','gols_total','pontos_alcancados','resultado', 'ano','adversario'], axis=1)

df = df[df['campeonato_ajustado'] == "Brasileirão"]
df = df.drop(['campeonato_ajustado'], axis=1)

In [9]:
df = df.reset_index(drop=True)

In [10]:
df = pd.get_dummies(df, drop_first=True)

### ANÁLISE DAS MELHORES VARIAVEIS

In [11]:
from sklearn.feature_selection import f_classif, f_regression

In [12]:
linear_dep = pd.DataFrame()

print("Quantidade de colunas numéricas restantes", len(df.select_dtypes(include='number').columns))

print("Quantidade de colunas categóricas restantes", len(df.select_dtypes(exclude='number').columns))

num_df = df[df.select_dtypes(include='number').columns]
num_df.fillna(0, inplace=True)

for col in num_df.columns:
    linear_dep.loc[col, 'pearson_corr'] = num_df[col].corr(num_df['publico_pagante'])
linear_dep['abs_pearson_corr'] = abs(linear_dep['pearson_corr'])

from sklearn.feature_selection import f_classif
for col in num_df.columns:
    mask = num_df[col].notnull()
    (linear_dep.loc[col, 'F'], linear_dep.loc[col, 'p_value']) = f_regression(pd.DataFrame(num_df.loc[mask, col]), num_df['publico_pagante'].loc[mask])

Quantidade de colunas numéricas restantes 54
Quantidade de colunas categóricas restantes 444


C:\Users\Dudu_\AppData\Local\Temp\ipykernel_19312\2590717999.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.fillna(0, inplace=True)


In [13]:
linear_dep.sort_values(by='abs_pearson_corr', ascending=False)

,pearson_corr,abs_pearson_corr,F,p_value
publico_pagante,1.000000,1.000000,1.797693e+308,0.000000e+00
publico_presente,0.905640,0.905640,1.236088e+03,5.718894e-103
renda_bruta,0.884847,0.884847,9.775794e+02,6.929364e-92
ocupacao_estadio,0.791552,0.791552,4.546747e+02,6.643991e-60
publico_pagante_ultimo_jogo_em_casa,0.465742,0.465742,7.506735e+01,4.191967e-16
capacidade_estadio,0.378166,0.378166,4.522280e+01,1.038897e-10
renda_bruta_ultimo_jogo_em_casa,0.334892,0.334892,3.423276e+01,1.406608e-08
aproveitamento_temporada_previo,-0.178122,0.178122,8.879852e+00,3.144950e-03
jogos_invencibilidade_previo,-0.152655,0.152655,6.465966e+00,1.155255e-02
dia_semana_int,0.129229,0.129229,4.602584e+00,3.281251e-02


In [14]:
linear_dep[linear_dep['p_value'] < 0.1].sort_values(by='abs_pearson_corr', ascending=False)

,pearson_corr,abs_pearson_corr,F,p_value
publico_pagante,1.000000,1.000000,1.797693e+308,0.000000e+00
publico_presente,0.905640,0.905640,1.236088e+03,5.718894e-103
renda_bruta,0.884847,0.884847,9.775794e+02,6.929364e-92
ocupacao_estadio,0.791552,0.791552,4.546747e+02,6.643991e-60
publico_pagante_ultimo_jogo_em_casa,0.465742,0.465742,7.506735e+01,4.191967e-16
capacidade_estadio,0.378166,0.378166,4.522280e+01,1.038897e-10
renda_bruta_ultimo_jogo_em_casa,0.334892,0.334892,3.423276e+01,1.406608e-08
aproveitamento_temporada_previo,-0.178122,0.178122,8.879852e+00,3.144950e-03
jogos_invencibilidade_previo,-0.152655,0.152655,6.465966e+00,1.155255e-02
dia_semana_int,0.129229,0.129229,4.602584e+00,3.281251e-02


In [15]:
from sklearn.feature_selection import f_classif

# Select categorical columns (excluding 'publico_pagante')
cat_cols = df.select_dtypes(exclude='number').columns

anova_cat_corr = pd.DataFrame(columns=['column', 'F_value', 'p_value'])

for col in cat_cols:
    # Only use rows without missing values in both columns
    mask = df[col].notnull() & df['publico_pagante'].notnull()
    if mask.sum() > 1 and df[col][mask].nunique() > 1:
        X = pd.DataFrame(df[col][mask])
        y = df['publico_pagante'][mask]
        # Convert categorical to codes for ANOVA
        X_encoded = X.apply(lambda x: x.astype('category').cat.codes)
        F, p = f_classif(X_encoded, y)
        anova_cat_corr = pd.concat([anova_cat_corr, pd.DataFrame({'column': [col], 'F_value': [F[0]], 'p_value': [p[0]]})], ignore_index=True)

anova_cat_corr.sort_values('F_value', ascending=False, inplace=True)
anova_cat_corr.reset_index(drop=True, inplace=True)
anova_cat_corr[anova_cat_corr['p_value'] < 0.05].sort_values('F_value', ascending=False)

C:\Users\Dudu_\AppData\Local\Temp\ipykernel_19312\3530545213.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  anova_cat_corr = pd.concat([anova_cat_corr, pd.DataFrame({'column': [col], 'F_value': [F[0]], 'p_value': [p[0]]})], ignore_index=True)
c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
c:\Us

,column,F_value,p_value
0,mes_5,inf,0.000000e+00
1,horario_17:00,inf,0.000000e+00
2,horario_19:30,inf,0.000000e+00
227,campeonato_20ª rodada do Campeonato Brasileiro...,5.106906e+08,2.420173e-121
228,campeonato_20ª rodada do Campeonato Brasileiro...,5.106906e+08,2.420173e-121
...,...,...,...
299,placar_2 × 0,2.784832e+00,8.163693e-04
300,periodo_noite,2.618504e+00,1.444273e-03
301,periodo_tarde,2.433930e+00,2.774132e-03
302,ano_2023,2.063084e+00,1.092250e-02


In [16]:
anova_cat_corr.F_value.sum()

np.float64(inf)

In [17]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

X = df.drop(['publico_pagante','data_formatada','Data','publico_presente','renda_bruta','ocupacao_estadio'], axis=1)
y = df['publico_pagante'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")


MAE: 9265.109386097301
MSE: 161032808.3110142
RMSE: 12689.87030315969
R²: 0.18260931968688965


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns


df_importance = pd.DataFrame(data=model.feature_importances_, index=X.columns, columns=['importance']).sort_values(by='importance', ascending=False)

df_importance

,importance
estadio_Mineirão,0.329512
renda_bruta_ultimo_jogo_em_casa,0.137556
ano_2024,0.059670
ano_2018,0.035172
precipitacao_total_lag2,0.033866
...,...
adversario_Avaí,0.000000
adversario_Bahia,0.000000
adversario_Botafogo,0.000000
adversario_Botafogo-SP,0.000000
